In [ ]:
import torch

# 3 by 3
x = torch.Tensor([[1, 2, 3], [4, 5, 6], [7, 8, 9]])
print(x)

print("shape of tensor : ", x.shape)
print("Type of tensor : ", x.dtype)
print("Allocated device : ", x.device)
print("demension : ", x.ndimension)

tensor([[1., 2., 3.],
        [4., 5., 6.],
        [7., 8., 9.]])
shape of tensor :  torch.Size([3, 3])
Type of tensor :  torch.float32
Allocated device :  cpu
demension :  <built-in method ndimension of Tensor object at 0x790c785b53d0>


In [ ]:
# Neural Network
import torch.nn as nn

# define model (2 floor, FC)

class simple_model(nn.Module):
  def __init__(self, input_size, hidden_size, num_classes):
    super(simple_model, self).__init__()
    self.fc1 = nn.Linear(input_size, hidden_size)   # first fc layer
    self.relu = nn.ReLU()                           # activation fun
    self.fc2 = nn.Linear(hidden_size, num_classes)  # second fc layer
    self.sigmoid = nn.Sigmoid()

  def forward(self, x):
    out = self.fc1(x)
    out = self.relu(out)
    out = self.fc2(out)
    return out


In [ ]:
import torchvision
from torchvision import transforms

# LOAD MNIST DATA
train_dataset = torchvision.datasets.MNIST(root='./data', train=True, transform=transforms.ToTensor(), download=True)
test_dataset = torchvision.datasets.MNIST(root='./data', train=False, transform=transforms.ToTensor(), download=True)

image, label = train_dataset[0]
print(image.size())   # torch.size(1,28,28)
print(label)          # 5

# Data Loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=64, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=64, shuffle=False)

100%|██████████| 9.91M/9.91M [00:00<00:00, 18.2MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 506kB/s]
100%|██████████| 1.65M/1.65M [00:00<00:00, 4.71MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 3.58MB/s]

torch.Size([1, 28, 28])
5


In [ ]:
# parameter setitng
input_size = 784    #28 * 28
hidden_size = 256
num_classes = 10
learning_rate = 1e-2

# initialization Model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = simple_model(input_size, hidden_size, num_classes).to(device)

# loss fun, optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [ ]:
# Learnig Loop
num_epoch = 10  # 데이터 셋의 데이터를 모두 사용해서 총 몇 번의 반복학습을 할 것인가?

for epoch in range (num_epoch):   # epochs 단위 학습
  for i, (images, labels) in enumerate(train_loader):   # batch 단위 학습
    images = images.reshape(-1, 28*28).to(device)   # fc학습을 위해 2d이미지를 1차원의 벡터로 변경 (batch_size,1,28,28) -> (batch_size, 784)
    labels = labels.to(device)

    #FeedFroward
    outputs = model(images)     # 모델로 부터 예측 값 얻기
    loss = criterion(outputs, labels)   # 손실 함수를 이용해 정답 라벨과 차이 정도 계산

    #BackPropagation
    optimizer.zero_grad()   # 모든 gradient 0으로 초기화
    loss.backward()         # 손실 함수 값 바탕으로 각 가중치의 gradient 계산
    optimizer.step()        # gradient 값을 바탕으로 가중치 업데이트

  print(f'Epoch [{epoch+1}/[num_epoch]], Loss : {loss.item():.4f}')

Epoch [1/[num_epoch]], Loss : 0.2630
Epoch [2/[num_epoch]], Loss : 0.0591
Epoch [3/[num_epoch]], Loss : 0.2820
Epoch [4/[num_epoch]], Loss : 0.1894
Epoch [5/[num_epoch]], Loss : 0.3335
Epoch [6/[num_epoch]], Loss : 0.1200
Epoch [7/[num_epoch]], Loss : 0.1425
Epoch [8/[num_epoch]], Loss : 0.2729
Epoch [9/[num_epoch]], Loss : 0.2063
Epoch [10/[num_epoch]], Loss : 0.2247


In [ ]:
# 테스트셋을 이용해 모델의 정확도 평가하기
with torch.no_grad():
  correct = 0
  total = 0
  for images, labels in test_loader :   #테스트 데이터 불러오기
    images = images.reshape(-1, 28*28).to(device)
    labels = labels.to(device)
    outputs = model(images)         # 학습된 모델로부터 테스트 데이터의 예측 값 얻기
    _, predicted = torch.max(outputs.data, 1)   # 가장 확률이 높은 숫자 값 가져오기
    total += labels.size(0)
    correct += (predicted == labels).sum().item()

  print(f'Accuracy of the network on the 10000 test images : {100 * correct / total}%')

Accuracy of the network on the 10000 test images : 94.9%
